In [1]:
import tensorflow as tf
from tensorflow import keras
#print(tf.keras.__version__)
#print(tf.__version__)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math as mt
import random as rng

from sklearn import preprocessing

def getColumns(size):
    columns = []
    for i in range(size):
        columns.append(i)
    return columns

directory = "./CMAPSSData/train_FD001.txt"
turbo = pd.read_csv(directory, header=None, sep=' ')
turbo = turbo.drop([26, 27], axis=1)

#for i in turbo.columns:
#    column = turbo[i]
#    print(i)
#    print(column.describe())
    
#print(turbo.values)

# Drop the input parameters to get just the sensors
turboSens = turbo.drop([0,1,4], axis=1)
turboSens.columns = getColumns(turboSens.shape[1])

# Get the cycles and useful sensors. 
turboUsefulSens = turboSens.drop([0, 9, 17, 18], axis=1)

min_max_scaler = preprocessing.MinMaxScaler()
norm_sens = min_max_scaler.fit_transform(turboUsefulSens)

turboCycles = turbo[1]
#print(turboCycles)

deteriorate = []
maxCycle = float(turboCycles.max())

currentCount = maxCycle
for i in range(turbo.shape[0] - 1, -1, -1):
    currentRow = turboCycles.loc[i]
    deteriorate.insert(0, currentCount)
    if currentRow == 1:
        currentCount = maxCycle
    else:
        currentCount -= 1

        
deteriorate = pd.DataFrame(deteriorate)

deteriorateSwap = []
for i in deteriorate[0]:
    deteriorateSwap.append(maxCycle - i)
deteriorate = pd.DataFrame(deteriorateSwap)


norm_sens = pd.DataFrame(norm_sens)
deteriorate = deteriorate.join(norm_sens, lsuffix='0', rsuffix='1')
deteriorate.columns = getColumns(deteriorate.shape[1])



In [2]:
array = deteriorate.to_numpy()

arrayCopy = array.tolist()

simulations = []

convCount = 10
convDiv = int(convCount/2)

count = len(array)

while(count > 0):
    cycles = []
    element = [-1,-1]
    while(element[0]!=0):
        element = arrayCopy.pop(0)
        cycles.append(element)
        count-=1
    simulations.append(cycles)
    
#for sim in simulations:
#    lenth = len(sim)
#    remainder = lenth%convCount
#    for i in range(remainder):
#        sim.pop(0)
    
#simsRecomb = []
#for sim in simulations:
#    for i in sim:
#        simsRecomb.append(i)

#print(len(simsRecomb))

In [3]:

tens = []
tensCount = []

for sim in simulations:
    for i in range(convDiv, len(sim) - convDiv):
        ten = []
        total = 0
        for j in range(-convDiv,convDiv):
            index = i + j
            element = sim[index].copy()
            total+= element.pop(0)
            ten.append(element)
        total/=convCount
        tensCount.append(total)
        tens.append(ten)

    
print(len(tens))
print(len(tensCount))

19631
19631


In [4]:
simsReorder = []
simsCycleReorder = []

length = len(tens)
for i in range(length):
    index = rng.randint(0, len(tens)-1)
    simsReorder.append(tens.pop(index))
    simsCycleReorder.append(tensCount.pop(index))
    
    
    
simsTrain = simsReorder[:15704]
targetTrain = simsCycleReorder[:15704]

simsTest = simsReorder[15704:]
targetTest = simsCycleReorder[15704:]


width = (turboUsefulSens.shape[1])
print(len(simsTrain[0][0]))

19


In [5]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(150, 3, activation='relu', input_shape=(convCount, width)),
    tf.keras.layers.Conv1D(300, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(6),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 8, 150)            8700      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 6, 300)            135300    
_________________________________________________________________
dropout (Dropout)            (None, 6, 300)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 4, 300)            270300    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 300)            0         
_________________________________________________________________
dense (Dense)                (None, 1, 64)             19264     
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              6

In [6]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='mse', 
              metrics=['mae', 'mse'])

In [7]:
model.fit(simsTrain, targetTrain, epochs=1000, verbose=1, validation_data=(simsTest,targetTest))

Train on 15704 samples, validate on 3927 samples
Epoch 1/1000
15704/15704 [==============================] - 13s 841us/sample - loss: 2254.2343 - mae: 34.7952 - mse: 2254.2341 - val_loss: 1726.9331 - val_mae: 30.6863 - val_mse: 1726.9332
Epoch 2/1000
15704/15704 [==============================] - 12s 764us/sample - loss: 1869.5691 - mae: 31.1781 - mse: 1869.5692 - val_loss: 1745.9394 - val_mae: 29.1524 - val_mse: 1745.9385
Epoch 3/1000
15704/15704 [==============================] - 12s 790us/sample - loss: 1806.5051 - mae: 30.4212 - mse: 1806.5062 - val_loss: 1612.3918 - val_mae: 28.2418 - val_mse: 1612.3918
Epoch 4/1000
15704/15704 [==============================] - 12s 777us/sample - loss: 1743.7009 - mae: 29.8643 - mse: 1743.7003 - val_loss: 1599.6393 - val_mae: 28.1368 - val_mse: 1599.6394
Epoch 5/1000
15704/15704 [==============================] - 12s 757us/sample - loss: 1726.5568 - mae: 29.6147 - mse: 1726.5570 - val_loss: 1628.2798 - val_mae: 27.9892 - val_mse: 1628.2799
Epoch 

KeyboardInterrupt: 

In [9]:
model.evaluate(simsTest,targetTest)

3927/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[84.6938531777013, 6.9929533, 84.693886]

In [10]:


print(model.predict([simsTest[0]]))
 
print(targetTest[0])

print(model.predict([simsTest[1]]))

print(targetTest[1])

[[[141.94702]]]
149.5
[[[75.580605]]]
71.5
